In [1]:
from sklearn.metrics import mean_squared_error
import pandas as pd

import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("taxi-exp-2")



2025/12/12 05:01:20 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/12 05:01:20 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/Mlops_Zoom_camp/02-Experiment Tracking/mlruns/1', creation_time=1765512987112, experiment_id='1', last_update_time=1765512987112, lifecycle_stage='active', name='taxi-exp-2', tags={}>

In [2]:
def data_read_pre_processing(file_path):
    
   
    #reading the data from the defined path
    df = pd.read_parquet(file_path)
    
    #Converting the columns to timestamp
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    
    #Calculating duration of the trip
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)
    
    #filetring the data 
    df = df[(df.duration>=1) & (df.duration<=60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [3]:
def preprocessor(df,dv):

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']#['PULocationID','DOLocationID']
    numerical = ['trip_distance']
    train_dicts = df[categorical+numerical].to_dict(orient = 'records')
    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [4]:
df = data_read_pre_processing("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")

In [5]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


run_id = '85d38da14cc6402fa082848aa0580bad'


#client.list_experiments()
client.download_artifacts(run_id = run_id, path = 'preprocessor', dst_path='.')

'/workspaces/Mlops_Zoom_camp/02-Experiment Tracking/preprocessor'

In [6]:
import pickle
import mlflow
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [7]:
X_test = preprocessor(df, dv)

In [8]:
target = "duration"
y_test = df[target].values

In [10]:
model_name = "nyc-taxi-regressor"
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 3.59 s, sys: 110 ms, total: 3.7 s
Wall time: 2.33 s


{'rmse': 6.308346956947805}

In [11]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 3.27 s, sys: 8.07 ms, total: 3.28 s
Wall time: 1.85 s


{'rmse': 6.308346956947805}

In [12]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_62956/1316468422.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1765514011998, current_stage='Production', deployment_job_state=None, description=None, last_updated_timestamp=1765515796597, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='85d38da14cc6402fa082848aa0580bad', run_link=None, source='models:/m-aab16fc7c9674283a9f812a87a18c476', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [ ]:
import mlflow
print("TRAINING MLflow URI:", mlflow.get_tracking_uri())
print("TRAINING Registry URI:", mlflow.get_registry_uri())
